In [0]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2

from keras.layers import Conv2D, MaxPooling2D, Convolution2D, Dropout, Dense, Flatten
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
img = np.load("./npy_files/img_train.npy")/255.
tar = np.load("./npy_files/tar_train.npy")

img_test = np.load("./npy_files/img_test.npy")/255.
tar_test = np.load("./npy_files/tar_test.npy")

In [0]:
input_shape = (28, 28, 1)
nb_classes = 47
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)

model = Sequential()
model.add(Convolution2D(nb_filters,
                        kernel_size,
                        padding='valid',
                        input_shape=input_shape,
                        activation='relu'))

model.add(Convolution2D(nb_filters,
                        kernel_size,
                        activation='relu'))

model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# model.summary()

In [0]:
batch_size = 256
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3, height_shift_range=0.08, zoom_range=0.08)
val_gen = ImageDataGenerator()

train_generator = gen.flow(img, tar, batch_size=batch_size)
test_generator = val_gen.flow(img_test, tar_test, batch_size=batch_size)

t1 = img.shape[0]//batch_size
t2 = img_test.shape[0]//batch_size

In [0]:
try:
  history = model.fit_generator(train_generator, steps_per_epoch=t1, epochs=20, validation_data=test_generator, validation_steps=t2)
except KeyboardInterrupt as e:
  pass

In [0]:
model.save("./models/model3.h5")

In [4]:
model = keras.models.load_model("./models/model3.h5")

print("------------\nTesting acc")
print(model.evaluate(img_test, tar_test, verbose=1))
print("------------\nTraining acc")
print(model.evaluate(img, tar, verbose=1))

18800/18800 [==============================] - 5s 259us/step
[0.2924460790702637, 0.8971808510638298]
------------
Training acc
112800/112800 [==============================] - 11s 95us/step
[0.19393558671764025, 0.921072695035461]


In [26]:
idx = np.random.randint(0, 18000)

image = np.reshape(img_test[idx], (1, 28, 28, 1))
prediction = model.predict_classes(image)

print("Prediction: %s"%prediction[0])
print("Actual val: %s"%np.argmax(tar_test[idx]))

cv2.imshow(image[0,:,:,0]*255.)

Prediction: 14
Actual val: 14


In [23]:
# CLASS-WISE PREDICTION RESULTS

Y_test = np.argmax(tar_test, axis=1)
y_pred = model.predict_classes(img_test)
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.89      0.72       400
           1       0.52      0.74      0.62       400
           2       0.93      0.88      0.90       400
           3       0.99      0.99      0.99       400
           4       0.97      0.95      0.96       400
           5       0.94      0.92      0.93       400
           6       0.97      0.95      0.96       400
           7       0.97      0.99      0.98       400
           8       0.95      0.97      0.96       400
           9       0.68      0.87      0.76       400
          10       0.97      0.99      0.98       400
          11       0.97      0.98      0.98       400
          12       0.98      0.96      0.97       400
          13       0.93      0.94      0.93       400
          14       0.98      0.99      0.99       400
          15       0.72      0.68      0.70       400
          16       0.93      0.97      0.95       400
          17       0.97    